# Song Embeddings 

In this notebook, we'll use human-made music playlists to learn song embeddings. We'll treat a playlist as if it's a sentence and the songs it contains as words. We feed that to the word2vec algorithm which then learns embeddings for every song we have. These embeddings can then be used to recommend similar songs. This technique is used by Spotify, AirBnB, Alibaba, and others.

## Importing packages and dataset

In [ ]:
import numpy as np
import pandas as pd
import gensim 
from gensim.models import Word2Vec
from urllib import request
import warnings
warnings.filterwarnings('ignore')

The playlist dataset is a text file where every line represents a playlist. That playlist is basically a series of song IDs. 

In [ ]:
# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as 
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:] 

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]


The `playlists` variable now contains a python list. Each item in this list is a playlist containing song ids. We can look at the first two playlists here:

In [ ]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

## Training the Word2Vec Model
Our dataset is now in the shape the the Word2Vec model expects as input. We pass the dataset to the model, and set the following key parameters:
 * **size**: Embedding size for the songs. 
 * **window**: word2vec algorithm parameter -- maximum distance between the current and predicted word (song) within a sentence
 * **negative**: word2vec algorithm parameter -- Number of negative examples to use at each training step that the model needs to identify as noise


In [ ]:
%%time
model = Word2Vec(playlists, size=32, window=20, negative=50, min_count=1, workers=4)

CPU times: user 2min 44s, sys: 274 ms, total: 2min 44s
Wall time: 1min 32s


The model is now trained. Every song has an embedding. We only have song IDs, though, no titles or other info. Let's grab the song information file.

## Song Title and Artist File
Let's load and parse the file containing song titles and artists

In [ ]:
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]

Now, `songs` is a list containing the id, title, and artist of every song in our datset. It looks like this:

In [ ]:
songs[:3]

[['0 ', 'Gucci Time (w\\/ Swizz Beatz)', 'Gucci Mane'],
 ['1 ', 'Aston Martin Music (w\\/ Drake & Chrisette Michelle)', 'Rick Ross'],
 ['2 ', 'Get Back Up (w\\/ Chris Brown)', 'T.I.']]

To simplify looking up song titles by ID, we'll define a pandas dataframe to hold song information.

In [ ]:
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [ ]:
songs_df.head()

,title,artist
id,,
0,Gucci Time (w\/ Swizz Beatz),Gucci Mane
1,Aston Martin Music (w\/ Drake & Chrisette Mich...,Rick Ross
2,Get Back Up (w\/ Chris Brown),T.I.
3,Hot Toddy (w\/ Jay-Z & Ester Dean),Usher
4,Whip My Hair,Willow


Pandas dataframes give us the ability to easily search through the columns of our dataset. We can look at the songs of a certain artist, for example.

In [ ]:
songs_df[songs_df.artist == 'Taylor Swift'].head()

,title,artist
id,,
2189,Mine,Taylor Swift
2253,Tim McGraw,Taylor Swift
2277,Love Story,Taylor Swift
2404,Picture to Burn,Taylor Swift
2413,Should've Said No,Taylor Swift


### Looking up songs by their IDs
Pandas also give us the ability to retrieve the information of multiple songs by passing their ids. Let's for example retrieve the info for songs number 1, 10, and 100.

In [ ]:
songs_df.iloc[[1,10,58]]

,title,artist
id,,
1,Aston Martin Music (w\/ Drake & Chrisette Mich...,Rick Ross
10,Shake It,Elephant Man
58,Love Is Blind,Eve


## Recommending Similar Songs
Let's now pick a song, and see what similar songs the model recommends

In [ ]:
songs_df.iloc[2172]

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object

In [ ]:
song_id = 2172

# Ask the model for songs similar to song #2172
model.wv.most_similar(positive=str(song_id))

[('5549', 0.9995133876800537),
 ('5586', 0.9984676837921143),
 ('5634', 0.9981622695922852),
 ('3167', 0.9977067708969116),
 ('2014', 0.9971413612365723),
 ('11502', 0.9970285892486572),
 ('1922', 0.9967612028121948),
 ('6658', 0.9965383410453796),
 ('2849', 0.996526837348938),
 ('2063', 0.9963899254798889)]

Let's look up the titles and artists of these songs:

In [ ]:
similar_songs = np.array(model.wv.most_similar(positive=str(song_id)))[:,0]
songs_df.iloc[similar_songs]

,title,artist
id,,
5549,November Rain,Guns N' Roses
5586,The Last In Line,Dio
5634,Mr. Brownstone,Guns N' Roses
3167,Unchained,Van Halen
2014,Youth Gone Wild,Skid Row
11502,Turbo Lover,Judas Priest
1922,One,Metallica
6658,(Bang Your Head) Metal Health,Quiet Riot
2849,Run To The Hills,Iron Maiden


Let's define a function that prints out both the song title and the recommendations based on it:


In [ ]:
def print_recommendations(song_id):
    print( songs_df.iloc[song_id] )
    similar_songs = np.array(model.wv.most_similar(positive=str(song_id)))[:,0]
    return  songs_df.iloc[similar_songs] 


## More Example Recommendations

### Paranoid Android - Radiohead

In [ ]:
print_recommendations(19563)

title     Paranoid Android
artist           Radiohead
Name: 19563 , dtype: object


,title,artist
id,,
18238,New Fang,Them Crooked Vultures
1986,Unsung,Helmet
2054,Guilty,Gravity Kills
12891,Month Of May,Arcade Fire
40245,Gold Gun Girls,Metric
34196,Two Against One (w\/ Jack White),Rome
10194,Drunken Lullabies,Flogging Molly
16378,The Royal We,Silversun Pickups
38877,No New Tale To Tell,Love And Rockets


### California Love - 2Pac

In [ ]:
print_recommendations(842)

title     California Love (w\/ Dr. Dre & Roger Troutman)
artist                                              2Pac
Name: 842 , dtype: object


,title,artist
id,,
12335,King Of The Dancehall,Beenie Man
5890,Low (w\/ T-Pain),Flo-Rida
5684,Sexy Can I (w\/ Yung Berg),Ray J
5698,Turnin' Me On (w\/ Lil Wayne),Keri Hilson
329,Stronger,Kanye West
23358,Aunque No Te Pueda Ver,Alex Ubago
26355,Hello (w\/ Dragonette),Martin Solveig
5710,Telephone (w\/ Beyonce),Lady Gaga
14025,Siete Horas,Bebe


### Billie Jean - Michael Jackson

In [ ]:
print_recommendations(3822)

title         Billie Jean
artist    Michael Jackson
Name: 3822 , dtype: object


,title,artist
id,,
12749,Wanna Be Startin' Somethin',Michael Jackson
4181,Kiss,Prince & The Revolution
4187,I Wanna Dance With Somebody (Who Loves Me),Whitney Houston
1506,The Way You Make Me Feel,Michael Jackson
3358,Maneater,Daryl Hall & John Oates
3924,Broken Wings,Mr. Mister
3357,Manic Monday,The Bangles
15660,Let The Music Play,Shannon
500,Don't Stop 'Til You Get Enough,Michael Jackson
